# 🫀 Interactive Heart Disease Risk Predictor

This notebook provides an **interactive interface** for predicting heart disease risk using a previously trained and saved model.  
It includes:
- Model loading utilities  
- User input collection  
- Interactive patient risk assessment  
- Batch processing for multiple patients  
- Risk reports with recommendations  


## 🔧 Import Required Libraries
We start by importing all the necessary libraries for:
- Model loading (`joblib`)
- Data handling (`numpy`, `pandas`)
- Date handling (`datetime`)
- File management (`os`)
``

In [67]:
import joblib
import os
import numpy as np
import pandas as pd
from datetime import datetime
risk_percentage=0

## 📥 Load Model and Components
This function loads the latest or a specific version of the trained model along with:
- Model object  
- Feature info  
- Label encoders (if any)  
- Metadata  

Saved models are stored inside the `saved_model/` directory.


In [68]:
def load_model_and_components(save_dir="s/content/drive/MyDrive/HDP/saved_model", timestamp=None):
    """
    Load the saved model and components

    Parameters:
    save_dir: directory containing saved files
    timestamp: specific timestamp to load (if None, loads latest)

    Returns:
    dict: loaded components
    """

    if not os.path.exists(save_dir):
        print(f"Directory {save_dir} does not exist!")
        return {}

    try:
        # If no timestamp specified, find the latest
        if timestamp is None:
            model_files = [f for f in os.listdir(save_dir) if f.startswith('heart_disease_model_') and f.endswith('.pkl')]
            if not model_files:
                print("No model files found!")
                return {}

            # Extract timestamps and find latest
            timestamps = []
            for f in model_files:
                try:
                    ts = f.replace('heart_disease_model_', '').replace('.pkl', '')
                    timestamps.append(ts)
                except:
                    continue

            if not timestamps:
                print("No valid timestamps found!")
                return {}

            timestamp = max(timestamps)  # Latest timestamp
            print(f"Loading latest model version: {timestamp}")

        loaded_components = {}

        # Load model
        model_path = os.path.join(save_dir, f"heart_disease_model_{timestamp}.pkl")
        if os.path.exists(model_path):
            loaded_components['model'] = joblib.load(model_path)
            print(f"Model loaded: {model_path}")

        # Load feature info
        feature_path = os.path.join(save_dir, f"feature_info_{timestamp}.pkl")
        if os.path.exists(feature_path):
            loaded_components['feature_info'] = joblib.load(feature_path)
            print(f"Feature info loaded: {feature_path}")

        # Load encoders
        encoders_path = os.path.join(save_dir, f"label_encoders_{timestamp}.pkl")
        if os.path.exists(encoders_path):
            loaded_components['encoders'] = joblib.load(encoders_path)
            print(f"Label encoders loaded: {encoders_path}")

        # Load metadata
        metadata_path = os.path.join(save_dir, f"model_metadata_{timestamp}.pkl")
        if os.path.exists(metadata_path):
            loaded_components['metadata'] = joblib.load(metadata_path)
            print(f"Model metadata loaded: {metadata_path}")

        print(f"Model loading completed! Version: {timestamp}")
        return loaded_components

    except Exception as e:
        print(f"Error loading model: {e}")
        return {}

## 👨‍⚕️ Patient Data Input Function
The following function interactively collects **20 patient features** used in training the model.  
It includes default values, validation checks, and ensures consistent feature ordering.


In [69]:
def get_user_input():
    """
    Collect patient information for all 20 features used in training

    Returns:
    list: patient data in the correct order for model prediction
    """
    print("\n" + "="*60)
    print("HEART DISEASE RISK ASSESSMENT - PATIENT DATA INPUT")
    print("="*60)
    print("Please enter the following patient information:")
    print("(Press Enter to use default/median values for optional fields)")

    patient_data = []

    # 1. Age
    while True:
        try:
            age_input = input("\n1. Patient's age (years): ")
            if age_input.strip() == "":
                age = 45  # default age
                print(f"   Using default age: {age}")
                break
            else:
                age = float(age_input)
                if age < 0 or age > 120:
                    print("   Please enter a realistic age (0-120)")
                    continue
                break
        except ValueError:
            print("   Please enter a valid number")
    patient_data.append(age)

    # 2. Gender
    while True:
        try:
            gender_input = input("\n2. Gender (Enter 0 for Female, 1 for Male): ")
            if gender_input.strip() == "":
                gender = 1  # default male
                print(f"   Using default: Male")
                break
            else:
                gender = int(gender_input)
                if gender not in [0, 1]:
                    print("   Please enter 0 for Female or 1 for Male")
                    continue
                break
        except ValueError:
            print("   Please enter 0 or 1")
    patient_data.append(gender)

    # 3. Blood Pressure
    while True:
        try:
            bp_input = input("\n3. Blood pressure in mmHg (normal: 90-140): ")
            if bp_input.strip() == "":
                blood_pressure = 120  # default
                print(f"   Using default: {blood_pressure} mmHg")
                break
            else:
                blood_pressure = float(bp_input)
                if blood_pressure < 50 or blood_pressure > 250:
                    print("   Please enter a realistic blood pressure (50-250)")
                    continue
                break
        except ValueError:
            print("   Please enter a valid number")
    patient_data.append(blood_pressure)

    # 4. Cholesterol Level
    while True:
        try:
            chol_input = input("\n4. Cholesterol level in mg/dL (normal: <200, high: >240): ")
            if chol_input.strip() == "":
                cholesterol = 200  # default
                print(f"   Using default: {cholesterol} mg/dL")
                break
            else:
                cholesterol = float(chol_input)
                if cholesterol < 100 or cholesterol > 600:
                    print("   Please enter a realistic cholesterol level (100-600)")
                    continue
                break
        except ValueError:
            print("   Please enter a valid number")
    patient_data.append(cholesterol)

    # 5. Exercise Habits
    print("\n5. Exercise habits:")
    print("   0 = Sedentary/No exercise")
    print("   1 = Light exercise (1-2 times/week)")
    print("   2 = Moderate exercise (3-4 times/week)")
    print("   3 = Heavy exercise (5+ times/week)")
    while True:
        try:
            exercise_input = input("   Enter exercise level (0-3): ")
            if exercise_input.strip() == "":
                exercise = 1  # default light exercise
                print(f"   Using default: Light exercise")
                break
            else:
                exercise = int(exercise_input)
                if exercise not in [0, 1, 2, 3]:
                    print("   Please enter a number between 0-3")
                    continue
                break
        except ValueError:
            print("   Please enter a number between 0-3")
    patient_data.append(exercise)

    # 6. Smoking
    while True:
        try:
            smoking_input = input("\n6. Smoking status (Enter 0 for Non-smoker, 1 for Smoker): ")
            if smoking_input.strip() == "":
                smoking = 0  # default non-smoker
                print(f"   Using default: Non-smoker")
                break
            else:
                smoking = int(smoking_input)
                if smoking not in [0, 1]:
                    print("   Please enter 0 for Non-smoker or 1 for Smoker")
                    continue
                break
        except ValueError:
            print("   Please enter 0 or 1")
    patient_data.append(smoking)

    # 7. Family Heart Disease
    while True:
        try:
            family_input = input("\n7. Family history of heart disease (Enter 0 for No, 1 for Yes): ")
            if family_input.strip() == "":
                family = 0  # default no
                print(f"   Using default: No family history")
                break
            else:
                family = int(family_input)
                if family not in [0, 1]:
                    print("   Please enter 0 for No or 1 for Yes")
                    continue
                break
        except ValueError:
            print("   Please enter 0 or 1")
    patient_data.append(family)

    # 8. Diabetes
    while True:
        try:
            diabetes_input = input("\n8. Diabetes status (Enter 0 for No, 1 for Yes): ")
            if diabetes_input.strip() == "":
                diabetes = 0  # default no
                print(f"   Using default: No diabetes")
                break
            else:
                diabetes = int(diabetes_input)
                if diabetes not in [0, 1]:
                    print("   Please enter 0 for No or 1 for Yes")
                    continue
                break
        except ValueError:
            print("   Please enter 0 or 1")
    patient_data.append(diabetes)

    # 9. BMI
    while True:
        try:
            bmi_input = input("\n9. BMI (Body Mass Index) - normal: 18.5-24.9, overweight: 25-29.9, obese: >30: ")
            if bmi_input.strip() == "":
                bmi = 25.0  # default
                print(f"   Using default BMI: {bmi}")
                break
            else:
                bmi = float(bmi_input)
                if bmi < 10 or bmi > 60:
                    print("   Please enter a realistic BMI (10-60)")
                    continue
                break
        except ValueError:
            print("   Please enter a valid number")
    patient_data.append(bmi)

    # 10. High Blood Pressure
    while True:
        try:
            hbp_input = input("\n10. History of high blood pressure (Enter 0 for No, 1 for Yes): ")
            if hbp_input.strip() == "":
                hbp = 0  # default no
                print(f"   Using default: No high BP history")
                break
            else:
                hbp = int(hbp_input)
                if hbp not in [0, 1]:
                    print("   Please enter 0 for No or 1 for Yes")
                    continue
                break
        except ValueError:
            print("   Please enter 0 or 1")
    patient_data.append(hbp)

    # 11. Low HDL Cholesterol
    while True:
        try:
            hdl_input = input("\n11. Low HDL cholesterol (good cholesterol <40 mg/dL for men, <50 for women)? (0=No, 1=Yes): ")
            if hdl_input.strip() == "":
                low_hdl = 0  # default no
                print(f"   Using default: Normal HDL")
                break
            else:
                low_hdl = int(hdl_input)
                if low_hdl not in [0, 1]:
                    print("   Please enter 0 for No or 1 for Yes")
                    continue
                break
        except ValueError:
            print("   Please enter 0 or 1")
    patient_data.append(low_hdl)

    # 12. High LDL Cholesterol
    while True:
        try:
            ldl_input = input("\n12. High LDL cholesterol (bad cholesterol >160 mg/dL)? (0=No, 1=Yes): ")
            if ldl_input.strip() == "":
                high_ldl = 0  # default no
                print(f"   Using default: Normal LDL")
                break
            else:
                high_ldl = int(ldl_input)
                if high_ldl not in [0, 1]:
                    print("   Please enter 0 for No or 1 for Yes")
                    continue
                break
        except ValueError:
            print("   Please enter 0 or 1")
    patient_data.append(high_ldl)

    # 13. Alcohol Consumption
    print("\n13. Alcohol consumption:")
    print("   0 = None")
    print("   1 = Light (1-2 drinks/week)")
    print("   2 = Moderate (3-7 drinks/week)")
    print("   3 = Heavy (>7 drinks/week)")
    while True:
        try:
            alcohol_input = input("   Enter alcohol consumption level (0-3): ")
            if alcohol_input.strip() == "":
                alcohol = 1  # default light
                print(f"   Using default: Light consumption")
                break
            else:
                alcohol = int(alcohol_input)
                if alcohol not in [0, 1, 2, 3]:
                    print("   Please enter a number between 0-3")
                    continue
                break
        except ValueError:
            print("   Please enter a number between 0-3")
    patient_data.append(alcohol)

    # 14. Stress Level
    print("\n14. Stress level (1-10 scale):")
    print("   1-3 = Low stress")
    print("   4-6 = Moderate stress")
    print("   7-10 = High stress")
    while True:
        try:
            stress_input = input("   Enter stress level (1-10): ")
            if stress_input.strip() == "":
                stress = 5  # default moderate
                print(f"   Using default stress level: {stress}")
                break
            else:
                stress = float(stress_input)
                if stress < 1 or stress > 10:
                    print("   Please enter a number between 1-10")
                    continue
                break
        except ValueError:
            print("   Please enter a number between 1-10")
    patient_data.append(stress)

    # 15. Sleep Hours
    while True:
        try:
            sleep_input = input("\n15. Average sleep hours per night (recommended: 7-9 hours): ")
            if sleep_input.strip() == "":
                sleep = 7.5  # default
                print(f"   Using default sleep hours: {sleep}")
                break
            else:
                sleep = float(sleep_input)
                if sleep < 2 or sleep > 16:
                    print("   Please enter realistic sleep hours (2-16)")
                    continue
                break
        except ValueError:
            print("   Please enter a valid number")
    patient_data.append(sleep)

    # 16. Sugar Consumption
    print("\n16. Daily sugar consumption:")
    print("   0 = Low (<25g/day)")
    print("   1 = Moderate (25-50g/day)")
    print("   2 = High (>50g/day)")
    while True:
        try:
            sugar_input = input("   Enter sugar consumption level (0-2): ")
            if sugar_input.strip() == "":
                sugar = 1  # default moderate
                print(f"   Using default: Moderate sugar consumption")
                break
            else:
                sugar = int(sugar_input)
                if sugar not in [0, 1, 2]:
                    print("   Please enter a number between 0-2")
                    continue
                break
        except ValueError:
            print("   Please enter a number between 0-2")
    patient_data.append(sugar)

    # 17. Triglyceride Level
    while True:
        try:
            trig_input = input("\n17. Triglyceride level in mg/dL (normal: <150, high: >200): ")
            if trig_input.strip() == "":
                triglycerides = 150  # default
                print(f"   Using default triglycerides: {triglycerides} mg/dL")
                break
            else:
                triglycerides = float(trig_input)
                if triglycerides < 50 or triglycerides > 1000:
                    print("   Please enter a realistic triglyceride level (50-1000)")
                    continue
                break
        except ValueError:
            print("   Please enter a valid number")
    patient_data.append(triglycerides)

    # 18. Fasting Blood Sugar
    while True:
        try:
            fbs_input = input("\n18. Fasting blood sugar in mg/dL (normal: <100, prediabetic: 100-125, diabetic: >125): ")
            if fbs_input.strip() == "":
                fbs = 95  # default normal
                print(f"   Using default fasting blood sugar: {fbs} mg/dL")
                break
            else:
                fbs = float(fbs_input)
                if fbs < 50 or fbs > 400:
                    print("   Please enter a realistic blood sugar level (50-400)")
                    continue
                break
        except ValueError:
            print("   Please enter a valid number")
    patient_data.append(fbs)

    # 19. CRP Level
    while True:
        try:
            crp_input = input("\n19. CRP level in mg/L (C-reactive protein - normal: <3.0, high: >3.0): ")
            if crp_input.strip() == "":
                crp = 2.0  # default normal
                print(f"   Using default CRP level: {crp} mg/L")
                break
            else:
                crp = float(crp_input)
                if crp < 0 or crp > 50:
                    print("   Please enter a realistic CRP level (0-50)")
                    continue
                break
        except ValueError:
            print("   Please enter a valid number")
    patient_data.append(crp)

    # 20. Homocysteine Level
    while True:
        try:
            homo_input = input("\n20. Homocysteine level in μmol/L (normal: 5-15, high: >15): ")
            if homo_input.strip() == "":
                homocysteine = 10.0  # default normal
                print(f"   Using default homocysteine level: {homocysteine} μmol/L")
                break
            else:
                homocysteine = float(homo_input)
                if homocysteine < 0 or homocysteine > 100:
                    print("   Please enter a realistic homocysteine level (0-100)")
                    continue
                break
        except ValueError:
            print("   Please enter a valid number")
    patient_data.append(homocysteine)

    print(f"\nAll 20 features collected successfully!")
    return patient_data

## 🤖 Prediction Function
Takes patient data and uses the trained model to predict:
- Risk probability (%)  
- Risk category (Low, Medium, High)  
- Confidence level of prediction  


In [70]:
def predict_patient_risk(model, patient_data, patient_name="Patient"):
    """
    Predict heart disease risk for a patient

    Parameters:
    model: trained machine learning model
    patient_data: list of 20 patient features in correct order
    patient_name: name/ID of patient

    Returns:
    dict: prediction results
    """
    # Convert to numpy array and reshape for prediction
    patient_array = np.array(patient_data).reshape(1, -1)

    # Get prediction probability
    risk_prob = model.predict_proba(patient_array)[0][1] * 100  # probability of class 1 (disease)

    # Get prediction
    prediction = model.predict(patient_array)[0]

    # Determine confidence level based on probability
    if risk_prob > 80 or risk_prob < 20:
        confidence = "High"
    elif risk_prob > 65 or risk_prob < 35:
        confidence = "Medium"
    else:
        confidence = "Low"

    return {
        'patient': patient_name,
        'risk_probability': risk_prob,
        'prediction': prediction,
        'confidence_level': confidence,
        'raw_data': patient_data
    }

## 📊 Display Risk Assessment
Formats and prints the prediction results with **risk levels and recommendations**.


In [71]:
def display_risk_assessment(result):
    """
    Display formatted risk assessment results

    Parameters:
    result: dict containing risk assessment results
    """
    print("\n" + "="*60)
    print("RISK ASSESSMENT RESULTS")
    print("="*60)

    # Risk level color coding (for text display)
    risk_prob = result['risk_probability']
    if risk_prob > 70:
        risk_indicator = "HIGH RISK"
    elif risk_prob > 30:
        risk_indicator = "MEDIUM RISK"
    else:
        risk_indicator = "LOW RISK"

    print(f"Patient: {result['patient']}")
    print(f"Risk Probability: {result['risk_probability']:.2f}%")
    print(f"Risk Category: {risk_indicator}")
    print(f"Confidence Level: {result['confidence_level']}")
    # print("------------------",risk_percentage,"-----------------")
    global risk_percentage
    risk_percentage+=result['risk_probability']   # logic to update
    

    # Recommendations based on risk level
    print(f"\nRecommendations:")
    if risk_prob > 70:
        print("  • URGENT: Seek immediate medical attention")
        print("  • Schedule comprehensive cardiac evaluation")
        print("  • Consider lifestyle modifications immediately")
        print("  • Monitor symptoms closely")
    elif risk_prob > 30:
        print("  • Schedule medical consultation within 2-4 weeks")
        print("  • Implement lifestyle changes (diet, exercise)")
        print("  • Regular monitoring of cardiovascular health")
        print("  • Consider preventive medications if recommended by doctor")
    else:
        print("  • Continue current preventive care routine")
        print("  • Maintain healthy lifestyle habits")
        print("  • Regular annual check-ups")
        print("  • Stay aware of cardiovascular risk factors")

    print(f"\nIMPORTANT: This is a predictive assessment only.")
    print(f"Always consult with healthcare professionals for medical decisions.")

## 🖥️ Interactive Menu System
The **main menu** for running:
- Single patient assessment  
- Batch assessment  
- Model info display  
- Load different model versions  


In [72]:
def interactive_risk_prediction(loaded_components):
    """
    Main interactive function for risk prediction
    """
    model = loaded_components.get('model')
    metadata = loaded_components.get('metadata')

    if not model:
        print("Error: No model found in loaded components!")
        return

    while True:
        print("\n" + "="*60)
        print("INTERACTIVE HEART DISEASE RISK PREDICTOR")
        print("="*60)
        if metadata:
            print(f"Model Version: {metadata.get('timestamp', 'Unknown')}")
            print(f"Training Date: {metadata.get('training_date', 'Unknown')}")
        print("="*60)
        print("1. New Patient Assessment")
        print("2. Batch Assessment (multiple patients)")
        print("3. View Model Information")
        print("4. Load Different Model Version")
        print("5. Exit")

        choice = input("\nSelect an option (1-5): ").strip()

        if choice == '1':
            # Single patient assessment
            try:
                patient_name = input("Enter patient name/ID: ").strip() or "Anonymous Patient"
                patient_data = get_user_input()
                result = predict_patient_risk(model, patient_data, patient_name)
                display_risk_assessment(result)

                # Ask if user wants to save results
                save_choice = input("\nSave results to file? (y/n): ").strip().lower()
                if save_choice == 'y':
                    save_results_to_file([result])

            except KeyboardInterrupt:
                print("\nAssessment cancelled.")
            except Exception as e:
                print(f"\nError during assessment: {e}")
                print("Please check your inputs and try again.")

        elif choice == '2':
            # Batch assessment
            try:
                num_patients = int(input("How many patients to assess? "))
                results = []

                for i in range(num_patients):
                    print(f"\n--- Patient {i+1} of {num_patients} ---")
                    patient_name = input("Enter patient name/ID: ").strip() or f"Patient {i+1}"
                    patient_data = get_user_input()
                    result = predict_patient_risk(model, patient_data, patient_name)
                    results.append(result)
                    display_risk_assessment(result)

                # Summary
                print(f"\n" + "="*60)
                print("BATCH ASSESSMENT SUMMARY")
                print("="*60)
                high_risk = sum(1 for r in results if r['risk_probability'] > 70)
                medium_risk = sum(1 for r in results if r['risk_probability'] > 30 and r['risk_probability'] <= 70)
                low_risk = sum(1 for r in results if r['risk_probability'] <= 30)

                print(f"High Risk Patients: {high_risk}")
                print(f"Medium Risk Patients: {medium_risk}")
                print(f"Low Risk Patients: {low_risk}")

                # Save results
                save_choice = input("\nSave all results to file? (y/n): ").strip().lower()
                if save_choice == 'y':
                    save_results_to_file(results)

            except ValueError:
                print("Please enter a valid number.")
            except KeyboardInterrupt:
                print("\nBatch assessment cancelled.")

        elif choice == '3':
            # Model information
            display_model_info(loaded_components)

        elif choice == '4':
            # Load different model version
            print("\nAvailable model versions:")
            save_dir = "saved_model"
            if os.path.exists(save_dir):
                model_files = [f for f in os.listdir(save_dir) if f.startswith('heart_disease_model_') and f.endswith('.pkl')]
                timestamps = []
                for f in model_files:
                    try:
                        ts = f.replace('heart_disease_model_', '').replace('.pkl', '')
                        timestamps.append(ts)
                    except:
                        continue

                if timestamps:
                    print("Available versions:")
                    for i, ts in enumerate(sorted(timestamps, reverse=True)):
                        print(f"  {i+1}. {ts}")

                    choice_ts = input("\nEnter version number or timestamp (or 'latest' for most recent): ").strip()

                    if choice_ts.lower() == 'latest':
                        new_components = load_model_and_components(save_dir)
                    elif choice_ts.isdigit() and 1 <= int(choice_ts) <= len(timestamps):
                        selected_ts = sorted(timestamps, reverse=True)[int(choice_ts)-1]
                        new_components = load_model_and_components(save_dir, selected_ts)
                    else:
                        new_components = load_model_and_components(save_dir, choice_ts)

                    if new_components and new_components.get('model'):
                        loaded_components.update(new_components)
                        print("Model loaded successfully!")
                    else:
                        print("Failed to load model.")
                else:
                    print("No model versions found.")
            else:
                print("No saved models directory found.")

        elif choice == '5':
            print("Thank you for using the Heart Disease Risk Predictor!")
            break

        else:
            print("Invalid choice. Please select 1-5.")

## 📂 Saving Prediction Results to File

To ensure that the outcomes of the heart disease risk assessment are not lost, the application saves all generated results into a CSV file.  

### 🔹 Purpose
The function is designed to take prediction results (such as patient details, calculated risk percentage, and suggested diet plan) and store them in a structured format. This makes it easy to review, analyze, or share the assessments later.

### 🔹 How It Works
- **Automatic File Naming**: Each file is saved with a timestamp in its name (e.g., `heart_disease_risk_assessment_20250906_123045.csv`).  
- **Data Organization**: Results are stored in a tabular format, where each row represents one user’s assessment.  
- **Reliability**: In case something goes wrong while saving, an error message is shown instead of silently failing.

### 🔹 Example Output
After running the function, a new CSV file is created in the working directory.  
It contains fields like:
- Patient name  
- Age  
- Predicted heart disease risk (as a percentage)  
- Risk category (Low / Moderate / High)  
- Suggested diet recommendations  

This file can be opened in Excel, Google Sheets, or directly loaded back into Python for further analysis.

### ✅ Key Benefit
Storing the results ensures that health predictions and personalized diet plans are **trackable over time**, supporting trend analysis, reporting, and better decision-making.


In [73]:
def save_results_to_file(results):
    """
    Save assessment results to a CSV file

    Parameters:
    results: list of result dictionaries
    """
    try:
        # Create filename with timestamp
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"heart_disease_risk_assessment_{timestamp}.csv"

        # Convert results to DataFrame
        results_df = pd.DataFrame(results)
        results_df.to_csv(filename, index=False)

        print(f"Results saved to: {filename}")

    except Exception as e:
        print(f"Error saving file: {e}")

## 🧠 Displaying Model Information

To provide transparency and clarity about the prediction system, the application includes a functionality to display detailed information about the **machine learning model** that is being used for heart disease risk assessment.  

### 🔹 Purpose
This step ensures that users (and developers) understand the key aspects of the trained model, such as:
- What kind of model is being used  
- When it was trained  
- The data and features it relies on  
- How predictions are categorized into different risk levels  

### 🔹 Information Displayed
1. **Model Metadata**  
   - Model type (e.g., Random Forest, Logistic Regression)  
   - Training date (when the model was last built)  
   - Version or timestamp for version control  
   - Number of features used for training  
   - Number of training samples used  
   - Target classes (e.g., No Heart Disease, Heart Disease)  

2. **Features Used in Prediction**  
   The model evaluates **20 key features**, including:  
   - Age, Gender, Blood Pressure, Cholesterol Level  
   - Lifestyle factors (Exercise, Smoking, Alcohol, Sleep, Stress)  
   - Medical factors (Diabetes, BMI, Triglycerides, Fasting Blood Sugar, CRP, Homocysteine, etc.)  
   These inputs collectively help in determining the likelihood of heart disease.

3. **Risk Categories**  
   Predictions are interpreted into meaningful categories for better understanding:  
   - **Low Risk** → 0–30% chance  
   - **Medium Risk** → 30–70% chance  
   - **High Risk** → above 70% chance  

### ✅ Key Benefit
By displaying this information, the system maintains **explainability and trust**, allowing healthcare professionals, researchers, or users to confidently rely on the model’s predictions while being aware of its scope and limitations.


In [74]:
def display_model_info(loaded_components):
    """
    Display information about the loaded model
    """
    print("\n" + "="*60)
    print("MODEL INFORMATION")
    print("="*60)

    metadata = loaded_components.get('metadata')
    feature_info = loaded_components.get('feature_info')

    if metadata:
        print(f"Model Type: {metadata.get('model_type', 'Unknown')}")
        print(f"Training Date: {metadata.get('training_date', 'Unknown')}")
        print(f"Version: {metadata.get('timestamp', 'Unknown')}")
        print(f"Number of Features: {metadata.get('n_features', 'Unknown')}")
        print(f"Training Samples: {metadata.get('n_samples_train', 'Unknown')}")
        print(f"Target Classes: {metadata.get('target_classes', 'Unknown')}")

    features = [
        "1. Age", "2. Gender", "3. Blood Pressure", "4. Cholesterol Level",
        "5. Exercise Habits", "6. Smoking", "7. Family Heart Disease", "8. Diabetes",
        "9. BMI", "10. High Blood Pressure", "11. Low HDL Cholesterol", "12. High LDL Cholesterol",
        "13. Alcohol Consumption", "14. Stress Level", "15. Sleep Hours", "16. Sugar Consumption",
        "17. Triglyceride Level", "18. Fasting Blood Sugar", "19. CRP Level", "20. Homocysteine Level"
    ]

    print(f"\nFeatures used in prediction:")
    for feature in features:
        print(f"  {feature}")

    print(f"\nRisk Categories:")
    print(f"  Low Risk: 0-30%")
    print(f"  Medium Risk: 30-70%")
    print(f"  High Risk: >70%")

## ⚡ Quick Prediction Mode

The **Quick Prediction** feature allows users to perform a **fast heart disease risk assessment** for a single patient without navigating through the full menu system. This makes it ideal for quick checks, testing, or one-time predictions.  

### 🔹 Workflow
1. **Load Model**  
   The function retrieves the trained machine learning model from the loaded components.  
   - If the model is missing, an error is shown.  

2. **User Input**  
   - The system prompts the user to enter the **patient’s name**.  
   - Next, the required **patient health parameters** are collected interactively through `get_user_input()`.  

3. **Risk Prediction**  
   - The collected patient data is passed to the model via the `predict_patient_risk()` function.  
   - The prediction result is generated (in terms of probability and risk category).  

4. **Display Results**  
   - The system outputs a clear and structured **risk assessment report** for the patient.  

5. **Return Result**  
   - The function returns the full result dictionary for later use (e.g., saving, integration with diet suggestions).  

### 🔹 Key Benefits
- **Fast and simple**: No need for a full UI or multiple steps.  
- **User-friendly**: Asks only for necessary patient data.  
- **Actionable**: Provides immediate heart disease risk level (Low, Medium, High).  

### ✅ Example Use Case
A healthcare professional quickly wants to check the **risk percentage for one patient** without going through the detailed system menus.  


In [75]:
def quick_prediction(loaded_components):
    """
    Quick single patient prediction without menu system
    """
    model = loaded_components.get('model')

    if not model:
        print("Error: No model found in loaded components!")
        return None

    print("Quick Heart Disease Risk Assessment")
    print("-" * 40)

    patient_name = input("Patient name: ").strip() or "Patient"
    patient_data = get_user_input()
    result = predict_patient_risk(model, patient_data, patient_name)
    display_risk_assessment(result)

    return result

## 🚀 Main Execution
Finally, we load the saved model and start the interactive prediction system.


In [76]:
def main():
    """
    Main function to load model and start interactive system
    """
    print("=" * 60)
    print("HEART DISEASE RISK PREDICTION SYSTEM")
    print("=" * 60)
    print("Loading saved model...")

    # Load the model
    loaded_components = load_model_and_components("saved_model")

    if not loaded_components or not loaded_components.get('model'):
        print("Error: Could not load the model!")
        print("Please ensure you have trained and saved a model first.")
        print("The saved model should be in the 'saved_model' directory.")
        return

    print("\nModel loaded successfully!")
    print("Features: Age, Gender, Blood Pressure, Cholesterol Level, Exercise Habits,")
    print("Smoking, Family Heart Disease, Diabetes, BMI, High Blood Pressure,")
    print("Low HDL Cholesterol, High LDL Cholesterol, Alcohol Consumption,")
    print("Stress Level, Sleep Hours, Sugar Consumption, Triglyceride Level,")
    print("Fasting Blood Sugar, CRP Level, Homocysteine Level")

    # Display model information if available
    metadata = loaded_components.get('metadata')
    if metadata:
        print(f"\nLoaded Model Information:")
        print(f"- Model Type: {metadata.get('model_type', 'Unknown')}")
        print(f"- Training Date: {metadata.get('training_date', 'Unknown')}")
        print(f"- Version: {metadata.get('timestamp', 'Unknown')}")

    # Start the interactive prediction system
    try:
        interactive_risk_prediction(loaded_components)
    except KeyboardInterrupt:
        print("\n\nSystem interrupted by user.")
        print("Thank you for using the Heart Disease Risk Prediction System!")
    except Exception as e:
        print(f"\nUnexpected error occurred: {e}")
        print("Please check your model files and try again.")
    finally:
        print("Goodbye!")

# Run the main function if script is executed directly
if __name__ == "__main__":
    main()

HEART DISEASE RISK PREDICTION SYSTEM
Loading saved model...
Loading latest model version: 20250906_075310
Model loaded: saved_model\heart_disease_model_20250906_075310.pkl
Feature info loaded: saved_model\feature_info_20250906_075310.pkl
Label encoders loaded: saved_model\label_encoders_20250906_075310.pkl
Model metadata loaded: saved_model\model_metadata_20250906_075310.pkl
Model loading completed! Version: 20250906_075310

Model loaded successfully!
Features: Age, Gender, Blood Pressure, Cholesterol Level, Exercise Habits,
Smoking, Family Heart Disease, Diabetes, BMI, High Blood Pressure,
Low HDL Cholesterol, High LDL Cholesterol, Alcohol Consumption,
Stress Level, Sleep Hours, Sugar Consumption, Triglyceride Level,
Fasting Blood Sugar, CRP Level, Homocysteine Level

Loaded Model Information:
- Model Type: RandomForestClassifier
- Training Date: 2025-09-06T07:53:10.871881
- Version: 20250906_075310

INTERACTIVE HEART DISEASE RISK PREDICTOR
Model Version: 20250906_075310
Training Date


Select an option (1-5):  1
Enter patient name/ID:  ss



HEART DISEASE RISK ASSESSMENT - PATIENT DATA INPUT
Please enter the following patient information:
(Press Enter to use default/median values for optional fields)



1. Patient's age (years):  23

2. Gender (Enter 0 for Female, 1 for Male):  1

3. Blood pressure in mmHg (normal: 90-140):  123

4. Cholesterol level in mg/dL (normal: <200, high: >240):  


   Using default: 200 mg/dL

5. Exercise habits:
   0 = Sedentary/No exercise
   1 = Light exercise (1-2 times/week)
   2 = Moderate exercise (3-4 times/week)
   3 = Heavy exercise (5+ times/week)


...

   Using default homocysteine level: 10.0 μmol/L

All 20 features collected successfully!

RISK ASSESSMENT RESULTS
Patient: ss
Risk Probability: 24.71%
Risk Category: LOW RISK
Confidence Level: Medium

Recommendations:
  • Continue current preventive care routine
  • Maintain healthy lifestyle habits
  • Regular annual check-ups
  • Stay aware of cardiovascular risk factors

IMPORTANT: This is a predictive assessment only.
Always consult with healthcare professionals for medical decisions.



Save results to file? (y/n):  n



INTERACTIVE HEART DISEASE RISK PREDICTOR
Model Version: 20250906_075310
Training Date: 2025-09-06T07:53:10.871881
1. New Patient Assessment
2. Batch Assessment (multiple patients)
3. View Model Information
4. Load Different Model Version
5. Exit



Select an option (1-5):  5


Thank you for using the Heart Disease Risk Predictor!
Goodbye!


## 💾 Saving Prediction Outcome with Pickle

To make the heart disease prediction results reusable across different parts of the project, the **outcome dictionary** is saved into a file using Python’s `pickle` module.  

### 🔹 Purpose
- Store the **calculated risk probability** in a serialized format.  
- Allow easy **reloading** of results without recalculating the prediction.  
- Enable **data sharing** between different modules (e.g., diet recommendation system).  

### 🔹 How It Works
1. The outcome is stored as a Python dictionary:  
   ```python
   outcome = {
       'risk_probability': risk_percentage
   }


In [77]:

outcome={
  'risk_probability': risk_percentage
  }
import pickle
with open("outcome.pkl", "wb") as f:
    pickle.dump(outcome, f)